In [1]:
import numpy as np

In [6]:
def qr_eigen(A, max_iter=100, tol=1e-12):
    n = A.shape[0]
    A_k = A.copy().astype(complex)
    Q_total = np.eye(n, dtype=complex)

    for _ in range(max_iter):
        Q, R = np.linalg.qr(A_k)
        A_k_new = R @ Q
        Q_total = Q_total @ Q

        if np.linalg.norm(A_k_new - A_k) < tol:
            break
        A_k = A_k_new

    eigvals = np.real(np.diag(A_k))
    return np.array(eigvals), Q_total



In [ ]:
def compute_svd(A):
    m, n = A.shape
    AAt = A @ A.conj().T

    eigvals, U = qr_eigen(AAt)
    idx = np.argsort(-eigvals)
    eigvals = eigvals[idx]
    U = U[:, idx]

    sigma = np.sqrt(np.clip(eigvals[:min(m, n)], 0, None))
    Sigma = np.zeros((m, n), dtype=complex)
    np.fill_diagonal(Sigma, sigma)


    V = np.zeros((n, n), dtype=complex)
    for i in range(min(m, n)):
        if sigma[i] > 1e-12:
            V[:, i] = (A.conj().T @ U[:, i]) / sigma[i]
            V[:, i] /= np.linalg.norm(V[:, i])  # Normalize
        else:
            V[:, i] = 0

    Vh = V.conj().T
    return U, Sigma, Vh


In [33]:
def RMSE(A, U, S, Vh):
    A_hat = U @ S @ Vh
    return np.sqrt(np.mean(np.abs(A - A_hat)**2))


In [34]:
A = np.random.randn(4, 4) + 1j * np.random.randn(4, 4)
U, S, Vh = compute_svd(A)
error = RMSE(A, U, S, Vh)
print("RMSE:", error)


RMSE: 1.3706890733646687e-14


In [30]:
A_hat = U @ S @ Vh

print("Original A:\n", A)
print("Reconstructed A_hat:\n", A_hat)
print("Error Matrix:\n", np.abs(A - A_hat))
print("Max Error:", np.max(np.abs(A - A_hat)))
print("RMSE:", RMSE(A, U, S, Vh))


Original A:
 [[-1.22  -0.3289j -0.1298+0.1817j  0.3109+0.1232j  0.9944+1.7279j]
 [ 0.5865-0.2827j  3.4612-0.6766j -0.8032-2.0955j  0.4351-1.1031j]
 [-0.2864-0.0031j -0.2244-0.2046j  1.248 +1.1733j  2.023 +0.3405j]
 [ 2.4848-1.6778j  0.9084-1.4392j  0.0806+2.693j   0.8702-2.5695j]]
Reconstructed A_hat:
 [[-0.4682-0.8598j  0.435 +1.1581j  1.2688-0.1482j -0.8857+0.9099j]
 [ 1.4261+1.0485j  1.1285+0.5944j -2.3599+1.3065j  1.6977+2.0943j]
 [-0.3373-1.1274j  1.5911+0.2181j  0.3537-1.6458j -0.7156+0.0841j]
 [ 2.3684-0.0781j  2.8304-1.3148j -1.8136-0.5573j  0.0287-2.7474j]]
Error Matrix:
 [[0.9204 1.128  0.9956 2.0504]
 [1.5739 2.6564 3.7412 3.4377]
 [1.1254 1.864  2.9576 2.7506]
 [1.604  1.926  3.7619 0.86  ]]
Max Error: 3.7619419103497895
RMSE: 2.303411381095242


In [31]:
AAt = A @ A.conj().T
AtA = A.conj().T @ A

eig1, _ = qr_eigen(AAt)
eig2, _ = qr_eigen(AtA)

print("Singular vals from AAt:", np.sort(np.sqrt(np.clip(eig1, 0, None)))[::-1])
print("Singular vals from AtA:", np.sort(np.sqrt(np.clip(eig2, 0, None)))[::-1])


Singular vals from AAt: [5.7667 4.3314 2.5848 0.4016]
Singular vals from AtA: [5.7667 4.3314 2.5848 0.4016]
